# GEFS reforecast wget cmd generator

This notebook generates qsub scripts to download GEFS reforecasts
* Source: https://noaa-gefs-retrospective.s3.amazonaws.com/GEFSv12/reforecast/

In [1]:
import os
import numpy as np
from datetime import datetime, timedelta

In [ ]:
import sys
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/')
from namelist import *

In [2]:
keys = ['c00', 'p01', 'p02', 'p03', 'p04']
# /Days:1-10
vars = ['apcp_sfc_{}_{}.grib2',]# 'cape_sfc_{}_{}.grib2', 'pwat_eatm_{}_{}.grib2', 
             #'tmp_2m_{}_{}.grib2', 'spfh_2m_{}_{}.grib2']

In [3]:
base = datetime(2002, 1, 1)
date_list = [base + timedelta(days=day) for day in range(365*18+4)]

## Fill-in-the-gap version

In [6]:
# check if files exist
date_list_fill = []

for dt in date_list:
    
    dt_str = datetime.strftime(dt, '%Y%m%d00')
    filename = camp_dir+'wget_GEFS_reforecast/{}_done.txt'.format(dt_str)
    
    if os.path.isfile(filename):
        continue;
    else:
        date_list_fill.append(dt)
        

In [8]:
L_fill = len(date_list_fill)
print('need to download {} files'.format(L_fill))
#date_list_fill = date_list_fill[::-1]

need to download 0 files


## Generate qsub scripts

In [9]:
N = int(L_fill/4) + 1

index = np.arange(0, L_fill, N)

for i_, i_start in enumerate(index):

    f = open(work_dir+'qsub/wget_reforecast_{:03d}.sh'.format(i_), 'w') 
    
    heads = '''
    #!/bin/bash -l
    
    #PBS -N wget_reforecast
    #PBS -A NAML0001
    #PBS -l walltime=23:59:59
    #PBS -l select=1:ncpus=4:mem=12GB
    #PBS -q casper
    #PBS -o wget_reforecast.log
    #PBS -e wget_reforecast.err
    
    cd {}wget_GEFS_reforecast/
    
    '''.format(camp_dir)
    
    print(heads, file=f)
    
    for i in range(i_start, i_start+N, 1):
        if i < L_fill:
            dt = date_list_fill[i]
            dt_str = datetime.strftime(dt, '%Y%m%d%H')
            year = dt.year
            
            for member in keys:
                for var in vars:
                    var_name = var.format(dt_str, member)
                    download_link = 'https://noaa-gefs-retrospective.s3.amazonaws.com/GEFSv12/reforecast/{}/{}/{}/Days%3A1-10/{}'.format(
                        year, dt_str, member, var_name)
                    
                    save_name = var_name
                    
                    print('wget -L -O {} {}'.format(save_name, download_link), file=f)
                    
            print('touch {}_done.txt'.format(dt_str), file=f)
    
    f.close()

f = open(work_dir+'qsub/wget_reforecast_all.sh', 'w')

for i_, i_start in enumerate(index):
    print('qsub wget_reforecast_{:03d}.sh'.format(i_), file=f)
    
f.close()